In [1]:
import requests
import time

In [2]:
# Scrape all at once to reduce temporal inconsistencies due to lag
stationInfoRequest = requests.get('https://peacehealthrides.com/opendata/station_information.json')
stationStatusRequest = requests.get('https://peacehealthrides.com/opendata/station_status.json')
freeBikeStatusRequest = requests.get('https://peacehealthrides.com/opendata/free_bike_status.json')

In [3]:
# Convert json responses to python-friendly dictionaries
stationInfo = stationInfoRequest.json()
stationStatus = stationStatusRequest.json()
freeBikeStatus = freeBikeStatusRequest.json()

In [4]:
# Extract info we really want from the dictionaries. Output is a list of Dictionaries
stationList = stationInfo.get('data').get('stations')
stationStatusList = stationStatus.get('data').get('stations')
freeBikeList = freeBikeStatus.get('data').get('bikes') 
# except for this one
freeBikeUpdated = freeBikeStatus.get('last_updated')

In [5]:
# turn stationList into a dictionary keyed by station_id
stationDict = {}
for station in stationList:
    stationDict[station.get('station_id')] = station

In [6]:
# Assemble output in csv format
outList = []
outList.append('station/bike id,name,lon,lat,available_bikes,available_docks,last_updated\n') 
for station in stationStatusList:
    stationID = station.get('station_id')
    stationName = stationDict.get(stationID).get('name').replace(",", " @")
    line = stationID + ',' + stationName + ',' + str(stationDict.get(stationID).get('lon')) + ',' + str(stationDict.get(stationID).get('lat')) + ',' + str(station.get('num_bikes_available')) + ',' + str(station.get('num_docks_available')) + ',' + time.strftime("%Y%m%d%H%M%S",time.localtime(station.get('last_reported'))) +'\n'
    outList.append(line)

for bike in freeBikeList:
    line = bike.get('bike_id') + ',' + bike.get('name') + ',' + str(bike.get('lon')) + ',' + str(bike.get('lat')) + ',1,0,' + time.strftime("%Y%m%d%H%M%S",time.localtime(freeBikeUpdated)) + '\n'
    outList.append(line)

In [7]:
# write the list to a csv file
outFileName = 'BikeShare - ' + time.strftime("%Y%m%d%H%M%S",time.localtime(freeBikeUpdated)) + '.csv'

with open(outFileName, 'w') as f:
    for line in outList:
        f.write(line)
f.close()